# Introduksjon til PyTorch
## Standardfunksjoner
For å bruke Pytoch må vi laste pakken `torch`. Numpy kommer også til å være praktisk å ha tilgjengelig.

In [1]:
import numpy as np
import torch

np.random.seed(1444)
torch.manual_seed(1444)

Torch har mye til felles med numpy, men stor forskjell på de to pakkene er at mens numpy bruker array-er bruker torch tensorer. Disse er ulike datatyper, men de oppfører seg ekstremt likt. Man kan også lette bytte fram og tilbake mellom tensorer og array-er med `.numpy()`og `torch.from_numpy()`.

In [2]:
x_array = np.array([1,2,3]) #Numpy array
x_tensor = torch.tensor([1,2,3]) #Tensor
print(x_array == x_tensor)

#Gå fra numpy til torch og fra torch til numpy
print(x_array == x_tensor.numpy()) #.numpy() tar deg fra torch til numpy
print(x_tensor == torch.from_numpy(x_array)) #torch.from_numpy tar deg fra numpy til torch

tensor([True, True, True])
[ True  True  True]
tensor([True, True, True])


Numpy og torch oppfører seg veldig likt og mange kommandoer er veldig like. 

In [3]:
#Lage tensorer
x = torch.tensor([[1,2], [3,4]]) #Lager en tensor variant av matrisen med 1,2 i første og 3,4 i andre rad
x = torch.tensor([[1,2],[3,4]], dtype = torch.float) #Lager en matrise som over, men sørger for at typen er satt til float
x = torch.ones(3) #Lager en (3,) tensor med 1 i alle entries
x = torch.zeros(3) #Lager en (3,) tensor med 0 i alle entries
x = torch.linspace(0,1,100) #Lager en (100,) tensor med entries mellom 0 og 1 som alle er like langt unna hverandre
x = torch.arange(0,1,0.01) #Lager en tensor med entries mellom 0 og 1 som alle har avstand 0.01 til hverandre
x = torch.rand(5) #Lager en vektor med entries trukket fra U[0,1]-fordelinga
x = torch.randn(2) #Lager en vektor med entries trukket fra N(0,1)-fordelinga

#shape og datatype
x.dtype #Datatype til en tensor
x.shape #Shape til en tensor

#Lin-alg funksjoner
A = torch.rand((2,2))
x = torch.rand(2)
A @ x #matrisemultiplikasjon
torch.matmul(A, x) #Alternativ syntaks for A @ x
x*x #Entrywise multiplikasjon
x+x #Entrywise addisjon
A.T #Transponering

#Vanlige funksjoner
torch.log(x) #naturlig logaritme
torch.exp(x) #eksponensialfunksjon
torch.sin(x) #sinus, tilsvarende for cos, tan osv
torch.sum(x) #summere entries
torch.mean(x) #Ta gjennomsnitt over entries
torch.diag(x) #Lag en matrise med passende dimensjoner og putt x på diagonalen
torch.diag(A) #Hent ut diagonalen til A

#Slicing og elementer
A[0,1] #Henter ut entry (0,1) fra A
A[:,0] #Henter ut første kollonne fra A
A[x>0.3,:] #Henter ut alle rader der korresponderende entry i x er større enn 0.3



tensor([[0.6478, 0.7097],
        [0.6743, 0.8946]])

Man kan også som regel sende tensorer inn til numpy funksjoner.

In [4]:
np.log(x)

/tmp/ipykernel_37221/1277889159.py:1: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  np.log(x)


tensor([-0.0892, -0.6188])

Man kan enkelt lage nye vektorer med samme dimensjon som en annen ved å legge til _like til mange funksjoner.

In [6]:
torch.ones_like(A)
torch.zeros_like(A)
torch.rand_like(A)
torch.randn_like(A)

tensor([[ 0.2763,  0.9976],
        [ 0.5432, -0.0929]])

Å slå sammen tensorer er litt knotete, men ikke noe mer enn i numpy.

In [7]:
A = torch.tensor([[1,1],[1,1]])
B = torch.zeros((3,2))
torch.cat([A,B], 0) #Legger B til under A
torch.cat([B.T,A], 1) #Legger A til til venstre for B.T

tensor([[0., 0., 0., 1., 1.],
        [0., 0., 0., 1., 1.]])

For at cat skal fungere, må dimensjonene matche. En 1-dim tensor av lengde d har formelt shape (d,) og må derfor reshapes for å kunne legges til en (d,d)-matrise

In [8]:
x = torch.zeros(2)
torch.cat([A,x.reshape(1,2)])

tensor([[1., 1.],
        [1., 1.],
        [0., 0.]])

## Automatisk derivering

Noe av det som gjør PyTorch så utrolig nyttig er at det bruker noe som heter automatisk derivering. Automatisk derivering er en måte å få datamaskinen til å regne ut den deriverte til funksjoner. Dette blir gjort på en analytisk måte, så svaret man får er **ikke** en tilnærming slik som i nummerisk derivering, men faktisk den sanne verdien av den deriverte.

Automatisk derivering fungerer ved å utnytte kjernereglen for alt den er verdt. Hvis en funksjon $f(x)$ kan skrives som en komposisjon av to andre funksjoner: $f(x) = g[h(x)]$, sier kjernereglen at $f'(x) = g'[h(x)]h'(x)$. Hvis man kjenner $g'[h(x)]$ og $h'(x)$ kan man derfor finne $f'(x)$ uten å regne på utrykket for $f$ ved å bruke kjernereglen. Dette er teoremet som får automatisk derivering til å fungere.

Enhver funksjon som er implementert på datamaskinen din er i bunn og grunn en lang komposisjon av enkle operasjoner som +, -, $\times$, / og ^. For eksempel kan $f(x) = 2x^2 + 3$ skrives som $f(x) = g\{h[k(x)]\}$ der $g(x) = x+3$, $h(x) = 2x$ og $k(x) = x^2$, et mer komplisert eksempel er $\log x$ som regnes ut av datamaskinen ved hjelp av en Taylor-tilnærming. Generelt kan vi skrive mer eller mindre alle funksjoner som $f(x) = g_n(g_{n-1}(...g_1(x)))$ der hver $g_i$ er en "enkel" operasjon. Pakkene som utfører automatisk derivering  har implementert utrykk for de deriverte til enkle operasjoner og holder styr på alle komposisjonene i funksjonene implementert på datamaskinen, slik at deres deriverte kan regnes ut automatisk ved hjelp av kjerneregelen!

**!**
Pytorch bruker pakken Autograd til å regne ut deriverte. Denne pakken kan man laste ned uten å laste ned PyTorch og den er mye enklere å bruke. Hvis kunn atomatisk derivering er av interesse, anbefaler jeg derfor å sjekke ut denne pakken.


For at automatisk derivering skal fungere, må datamaskinen holde styr på alle operasjonene som brukes for å regne ut f(x). Vi må fortelle maskinen at den skal gjøre dette. Alle tensorer har en egenskap `requires_grad` som forteller PyTorch hvorvidt den skal holde styr på alt som gjøres med tensoren eller ikke. Hvis vi vil derivere med hensyn på en tensor, må vi derfor sette denne egenskapen til `True` for denne tensoren 


Automatisk derivering fungerer bare for å regne ut verdien av den deriverte i hvert punkt. Det er ikke symbolsk derivering som prøver å evaluere utrykk og gi en formel for den deriverte (slik som i wolfram alpha).

In [5]:
x = torch.ones(3, requires_grad = True) #Sett argumentet til True når tensoren lages
x = torch.ones(3)
x.requires_grad = True #Sett argumentet til False etterpå

Den deriverte til en funksjon kan regnes ut ved hjelp av `torch.autograd.grad`. Denne funksjonen returnerer et tuppel med svaret i første argument og ingenting i det andre.

In [6]:
#Definerer en funksjon å derivere
def f(x):
    return torch.sin(2*torch.pi*x) + torch.log(x)

x = torch.ones(1, requires_grad = True) #Vi deriverer i x = 1

#Regn ut den deriverte med automatisk derivering
auto_diff = torch.autograd.grad(outputs=f(x),
                                inputs = x)
#Regn ut den deriverte analytisk
analytical_diff = torch.tensor([2*torch.pi*torch.cos(2*torch.pi*x) + 1/x])

#Print svar
print(auto_diff)
print(analytical_diff)

(tensor([7.2832]),)
tensor([7.2832])


Hvis $f$ er vektorvaluert er $f'(x)$ en matrise og ikke en vektor. Da er det Jacobimatrisen man ønsker å regne ut. Dette kan man gjøre ved å bruke `torch.autograd.functional.jacobian`. Det er også en funksjon for å regne ut Hessematriser i `torch.autograd.functional`.

In [7]:
#Regner ut Hesse matrisen til f
torch.autograd.functional.hessian(f,
                                  inputs = x)

#Regn ut en Jacobimatrise av en vektorvaluert funksjon
x = torch.ones(2, requires_grad=True)
def f(x):
    return torch.sin(2*torch.pi*x) + torch.log(x)

torch.autograd.functional.jacobian(f,
                                   inputs = x)

tensor([[7.2832, 0.0000],
        [0.0000, 7.2832]])

### Eksempel
Automatisk derivering er implementert i PyTorch for å sørge for at modelltilpassingene av svært kompliserte modeller som nevrale nett går fort, men det kan være veldig praktisk for klassisk statistikk også, da det kan brukes for å gjøre optimeringsalgoritmer raskere og for å enkelt regne ut Fisher-matriser. Tettheter og lignende for mange vanlige statistiske modeller kan finnes på [PyTorch sine sider](https://pytorch.org/docs/stable/distributions.html#module-torch.distributions).

In [8]:
import scipy.stats as stats
from scipy.optimize import minimize

#Simmuler data fra Gamma(2,3)
np.random.seed(1444)
n = 50
x = torch.from_numpy(stats.gamma.rvs(size = n, a = 1, scale = 2))

#Definerer log-likelihood
def neg_log_lik(theta):
    dist = torch.distributions.gamma.Gamma(theta[0], theta[1])
    return -torch.sum(dist.log_prob(x))

#Definerer deriverte og hessematriser
jac = lambda x: torch.autograd.functional.jacobian(neg_log_lik, torch.from_numpy(x))
hess = lambda x: torch.autograd.functional.hessian(neg_log_lik, torch.from_numpy(x))

#Finner ML-estimat
mini = minimize(neg_log_lik,
                x0 = torch.ones(2),
                jac = jac,
                hess = hess,
                method = "Newton-CG")

#Tilnærmet varians
J = hess(mini.x)/n #Tilnærmet Fishermatrise
print("Tilnærmet varians for ML-estimator er:")
print(np.linalg.inv(J))


Tilnærmet varians for ML-estimator er:
tensor([[1.0633, 0.6630],
        [0.6630, 0.7405]])


Scriptet over tilpasser en gamma-fordeling til simmulert data og regner ut Fisher-matrisen uten at deriverte må tilnærmes eller regnes ut for hånd!

### Alternativ syntaks
Ovenfor har vi brukt pakken autograd (som er inkludert i PyTorch) direkte til å finne deriverte. Denne måten å gjøre ting på er relativt rett fram, men krever en del notasjon. PyTorch lar deg derfor regne ut deriverte på en enda enklere måte.

Hvis man setter argumentet `requires_grad` til en tensor `x` til `True`, vil PyTorch automatisk holde styr på alt som blir gjort med denne tensoren og deriverte kan hentes ut fortløpende. For at dette skal fungerer må vi imidlertid først fortelle PyTorch hva vi vil derivere med hensyn på `x`. Dette gjør man ved å kalle på `.backward()` som regner ut alle deriverte til tensoren man bruker den på. Gradientene kan hentes ut ved å hente ut `.grad` fra den tensoren man deriverer med hensyn på.

In [9]:
x = torch.ones(1, requires_grad=True)
y = 3*x**2

x.grad #Vi har ikke bedt PyTorch regne ut gradienter ennå, så denne er tom

#Regn ut gradienter
y.backward()

x.grad #Nå er partila y/partial x innerholdt i x

tensor([6.])

Hvis vi gjør flere ting med `x` trenger, men bare kaller `.backward()` på en av operasjonene blir bare denne operasjonen derivert

In [10]:
x = torch.ones(1, requires_grad=True)
y = 3*x**2
z = torch.sin(x)

#Regn ut gradienter
y.backward()

x.grad #Nå er partila y/partial x innerholdt i x

tensor([6.])

Vi kan regne ut deriverte med hensyn på flere variabler.

In [23]:
x = torch.ones(1, requires_grad=True)
z = torch.zeros(1, requires_grad=True)
y = 3*x**2 + torch.sin(z)

#Regn ut gradienter
y.backward()

print(x.grad)
print(z.grad)

tensor([6.])
tensor([1.])
